# Check valid switching states

**Author:** Manuela Linke, HTWG Konstanz 

**Date:** 14.04.2024 

**Summary:** This Script checks the valid switching states pf all possible combinations of the states of the controllable line switches, where all buses are supplied, and saves them in a "allowed_switches.npy"-file. It is also possible to allow for only radial grids.

**License:** Copyright 2024 Manuela Linke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import networkx as nx
import itertools
import pandapower as pp
import pandas as pd 
import numpy as np

## Checking for unsupplied nodes

In [2]:
filename="allowed_switches"
net=pp.from_pickle("cossmic_grid.p")

In [3]:
def is_supplied(net):
    return len(pp.topology.unsupplied_buses(net)) == 0

In [4]:
is_supplied(net)

True

## Checking for meshing in the network

In [5]:
def is_radial(net):
    mg = pp.topology.create_nxgraph(net, multi=False)
    cycles = list(nx.simple_cycles(mg))
    
    # If there are any cycles, the grid is meshed
    if cycles:
       return False # Meshing found
    else:
       return True  # No meshing, network is radial

In [6]:
is_radial(net)

True

## Analyse switch positions
The function analyze_switch_positions iterates over all possible switching states and determines the valid switching states, where the grid is radial and all buses are supplied.

In [7]:
# Use this function if you want to reduce the possible number of outputs
def analyze_switch_positions_only_3(net):
    net.switch = net.switch[net.switch['et'] == "l"]
    net.switch.closed = True
    switch_positions = {}
    for i, (s1, s2, s3) in enumerate(itertools.combinations(net.switch.index,3)):
        net.switch.loc[[s1, s2, s3], 'closed'] = False
        supplied = is_supplied(net)
        radial = is_radial(net)
        valid = supplied and radial 
        switch_positions[i] = {"supplied": supplied, "radial": radial,
                               "valid": valid, "switches": [s1, s2, s3]}
        net.switch.closed = True    
    return pd.DataFrame.from_dict(switch_positions).T

In [8]:
from itertools import combinations

def analyze_switch_positions(net):
    allowed_sw = []
    net.switch = net.switch[net.switch['et'] == "l"]
    sw = net.switch.index
    sw_comb = sum([list(map(list, combinations(sw, i))) for i in range(len(sw) + 1)], [])
    net.switch.closed=True
    switch_positions = {} 
    for i, comb in enumerate(sw_comb[0:len(sw_comb)]):
        net.switch.loc[comb, 'closed'] = False
        supplied = is_supplied(net)
        radial = is_radial(net)
        valid = supplied and radial 
        switch_positions[i] = {"supplied": supplied, "radial": radial,
                               "valid": valid, "switches": comb, "nr_closed_switches": sum(net.switch.closed) }
        if supplied:
            allowed_sw.append(net.switch['closed'].values)
        net.switch.closed = True

    return pd.DataFrame.from_dict(switch_positions).T, allowed_sw

In [9]:
sw, allowed_sw = analyze_switch_positions(net)

In [10]:
print("number of switching states: %u" % len(sw.index))
print("number of switching state with unsupplied buses: %u" % len(sw[sw.supplied == False]))
print("Valid switching states: %u"  %  (len(sw[sw.supplied])))

number of switching states: 4096
number of switching state with unsupplied buses: 3942
Valid switching states: 154


In [11]:
#Testing function
for switches in range(len(allowed_sw)):
    net.switch.loc[net.switch.et == 'l', 'closed'] = allowed_sw[switches] 
    supplied = len(pp.topology.unsupplied_buses(net)) == 0
    if (supplied == False):
        print("The following buses are unsupplied:", pp.topology.unsupplied_buses(net))
        net.switch.loc[net.switch.et == 'l', 'closed'] = allowed_sw[switches]       

In [12]:
directory = ""
filename = directory + "allowed_switches"
np.save(filename, allowed_sw)